In [1]:
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from src.inference import *
from src.misc import *
from src.models.predictors import *
from src.plots import *
from src.train import *
from pytorch_lightning.loggers import MLFlowLogger
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader, TensorDataset

KeyboardInterrupt: 

In [ ]:
d=5

X, _ = generate_synthetic_data_for_seq2one(32000,d,False)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

window_size = 10
X_in, Y_out = make_sequence_to_one(X_scaled, window_size)



In [ ]:
(torch.tensor(X_in[0]).unsqueeze(0).numpy()).astype(np.float32)

In [ ]:
import mlflow.pytorch


mlflow.pytorch.autolog(log_models=False)
mlflow.set_experiment("AnomalyDetection")

train_dataset = TensorDataset(torch.tensor(X_in[:700], dtype=torch.float32),
                              torch.tensor(Y_out[:700], dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# mlf_logger = MLFlowLogger(experiment_name="AnomalyDetection", run_name="LSTM-Predictor")
# Enable autologging with Lightning


checkpoint_callback = ModelCheckpoint(
    monitor="train_loss",
    save_top_k=1,
    mode="min",
)

model = SequenceToOneLSTM(input_dim=d, hidden_dim=32, lr=1e-3)

trainer = pl.Trainer(
    max_epochs=50,
    #logger=mlf_logger,
    callbacks=[checkpoint_callback],
    log_every_n_steps=1
)
with mlflow.start_run(run_name="LSTM-Predictor") as run:
    
    trainer.fit(model, train_loader)
    
    with torch.no_grad():
        input_example=(torch.tensor(X_in[0:1]).numpy().astype(np.float32))
        mlflow.pytorch.log_model(model,input_example=input_example)
mlflow.pytorch.autolog(disable=True)



In [ ]:
class SequenceToOneLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, input_dim)

    def forward(self, x):  # x: [B x T x D]
        _, (h_n, _) = self.lstm(x)   # h_n: [1 x B x H]
        out = self.fc(h_n.squeeze(0))  # [B x D]
        return out

model_base= SequenceToOneLSTM(input_dim=d, hidden_dim=32)

In [ ]:
print(model_base)

In [ ]:
train_predictor(model_base, torch.tensor(X_in[:700], dtype=torch.float32), torch.tensor(Y_out[:700], dtype=torch.float32), epochs=50, lr=1e-3, run_name="predictor")